In [6]:
%pip uninstall -y sagemaker
%pip install sagemaker==2.248.1

Found existing installation: sagemaker 2.248.1
Uninstalling sagemaker-2.248.1:
  Successfully uninstalled sagemaker-2.248.1
Note: you may need to restart the kernel to use updated packages.
  Using cached sagemaker-2.248.1-py3-none-any.whl.metadata (17 kB)
Using cached sagemaker-2.248.1-py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
which pip
pip --version
pip show sagemaker
conda list sagemaker

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ which pip                                                                                        │
│       ▲                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [1]:
import sagemaker
print(sagemaker.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
2.248.1


In [4]:
import sagemaker.processing as proc
print([n for n in dir(proc) if "NotebookProcessor" in n])

[]


In [3]:
%pip install --upgrade sagemaker

Note: you may need to restart the kernel to use updated packages.


In [7]:
from sagemaker.processing import NotebookProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline import Pipeline

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 from sagemaker.processing import NotebookProcessor, ProcessingInput, ProcessingOutput        │
│   2 from sagemaker.workflow.steps import ProcessingStep                                          │
│   3 from sagemaker.workflow.pipeline import Pipeline                                             │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'NotebookProcessor' from 'sagemaker.processing' 
(/opt/conda/lib/python3.12/site-packages/sagemaker/processing.py)

In [4]:
sess = Session()
role = sagemaker.get_execution_role()
region = sess.boto_region_name
bucket = "experimento-lucas-barbosa"

print(f"Role: {role}")
print(f"Region: {region}")
print(f"Bucket: {bucket}")

Role: arn:aws:iam::657444906686:role/service-role/AmazonSageMaker-ExecutionRole-20250708T150740
Region: sa-east-1
Bucket: experimento-lucas-barbosa


In [8]:
# Step 1: 00_config.ipynb
step_config = ProcessingStep(
    name="ConfigStep",
    processor=notebook_processor,
    code="00_config.ipynb",
    inputs=[],  # Adicione inputs se necessário
    outputs=[]
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 # Step 1: 00_config.ipynb                                                                    │
│   2 step_config = ProcessingStep(                                                                │
│   3 │   name="ConfigStep",                                                                       │
│ ❱ 4 │   processor=notebook_processor,                                                            │
│   5 │   code="00_config.ipynb",                                                                  │
│   6 │   inputs=[],  # Adicione inputs se necessário                                              │
│   7 │   outputs=[]                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'notebook_processor' is not defined